# Fine Tune

**Reference:** https://neptune.ai/blog/hugging-face-pre-trained-models-find-the-best

In [1]:
# !pip install -r requirements.txt

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Load Model

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")

In [4]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [5]:
sentence = "开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快"

tokenizer.src_lang = 'zh_CN'
tokenizer.tgt_lang = 'en_XX'

encoded = tokenizer(sentence, return_tensors="pt").to(device)
model = model.to(device)
generated_tokens = model.generate(**encoded)
decoded = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

print(decoded)

If the air conditioner is on, the flight resumes too quickly, especially in winter when the weather is cold. If the air conditioner is not on, the flight resumes faster as soon as the weather is cold


## Preprocess Data

### Load Data

In [6]:
import pandas as pd
from datasets import Dataset

In [7]:
df_1 = pd.read_excel('../data/trans/2023_NCVQS_text.xlsx', sheet_name=0)
df_1.rename(columns = {'Detail complains breakdown  (Chinese)': 'Chinese', 'Translation': 'English'}, inplace=True)

df_2 = pd.read_excel('../data/trans/2023_NCVQS_text.xlsx', sheet_name=1)

zh_columns = df_2.columns[:4]
en_columns = df_2.columns[4:]

df_2['Chinese'] = df_2[zh_columns].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
df_2['English'] = df_2[en_columns].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

df_2.drop(columns=en_columns, inplace=True)
df_2.drop(columns=zh_columns, inplace=True)

df = pd.concat([df_1, df_2], axis = 0)
df = df.dropna()
df.drop_duplicates(keep='first', subset='Chinese', inplace=True)

In [8]:
print("Length: ", len(df), "\n Num of NaN: ", df.isna().sum().sum())

Length:  447 
 Num of NaN:  0


In [9]:
df.head()

,Chinese,English
0,开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快,"In the case of turning on the air conditioner,..."
1,车机流畅度差，容易卡死机，车机系统，启动载入很慢，换挡杆前的车机，使用任何功能都有概率死机，...,"The smoothness of the IHU is poor, easy to jam..."
2,整车的悬架系统，在过减速带时，速度在20码以下，但是车身的抖动还是很厉害，舒适性为第一的，美...,"The suspension system of the whole car, when c..."
3,大众车的通病，车子的隔音效果不太理想，车速在90码以上，车内的胎噪声就很明显了，必须把音量调...,"The common problem of Volkswagen, the sound in..."
4,车辆外观很不错，但是车标在晚上不能发亮，要是可以发亮的话会更拉风一点,"The appearance of the vehicle is very good, bu..."


In [10]:
tmp = df.copy()

### Transform Data

In [11]:
df = df.assign(translation=df.apply(lambda row:{'zh': row['Chinese'], 'en': row['English']}, axis=1))
df.reset_index(inplace=True)
df.drop(['index', 'Chinese', 'English'], axis=1, inplace=True)

In [12]:
df.head()

,translation
0,{'zh': '开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一...
1,{'zh': '车机流畅度差，容易卡死机，车机系统，启动载入很慢，换挡杆前的车机，使用任何功...
2,{'zh': '整车的悬架系统，在过减速带时，速度在20码以下，但是车身的抖动还是很厉害，舒...
3,{'zh': '大众车的通病，车子的隔音效果不太理想，车速在90码以上，车内的胎噪声就很明显...
4,"{'zh': '车辆外观很不错，但是车标在晚上不能发亮，要是可以发亮的话会更拉风一点', '..."


In [13]:
df.iloc[0][0]

{'zh': '开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快',
 'en': "In the case of turning on the air conditioner, the electric range drops too fast, especially when the weather is cold in winter, if you don't turn on the air conditioner, the weather freezes, the battery life will fall faster."}

In [14]:
print("Dimension: ", df.shape)

Dimension:  (447, 1)


In [15]:
dataset = Dataset.from_pandas(df, split='train')
dataset = dataset.train_test_split(test_size=0.05)

In [16]:
print(type(dataset['train']))
print(dataset['train'][:1])

<class 'datasets.arrow_dataset.Dataset'>
{'translation': [{'en': 'Good quality, trouble-free, good service, pre-sale and after-sales are enthusiastic. Good power, energy saving, fast speed. Strong power, starting, acceleration, climbing are all powerful. No', 'zh': '质量好，无故障，服务好，售前售后都热情 动力好，节能，提速快 动力强，起步，加速，爬坡都动力足 无'}]}


In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 424
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 23
    })
})

### Tokenize Datasets

In [18]:
prefix = "" #for mBART and MarianMT
max_input_length = 512
max_target_length = 512

source_lang = "zh"
target_lang = "en"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
   
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/424 [00:00<?, ? examples/s]

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

## Train and Fine-tune the Model

### Model Setup

In [19]:
from transformers import Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer
import evaluate

In [20]:
from datetime import datetime

# Get the current date
today = datetime.now()

# Format the date as "DDMMYYYY"
formatted_date = today.strftime("%d%m%Y")

print(formatted_date)

09022024


In [21]:
model_name = f"mbart-finetuned-cn-to-en-auto"
model_path = f"../models/{model_name}"

batch_size = 4

args = Seq2SeqTrainingArguments(
   output_dir=model_path,
   evaluation_strategy="epoch",
   learning_rate=2e-5,
   per_device_train_batch_size=batch_size,
   per_device_eval_batch_size=batch_size,
   weight_decay=0.01,
   save_total_limit=3,
   num_train_epochs=2,
   predict_with_generate=True,
)

In [22]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) # default setting

In [23]:
import numpy as np
import evaluate

metric = evaluate.load("sacrebleu")
meteor = evaluate.load('meteor')

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    meteor_result = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result = {'bleu' : result['score']}
    result["gen_len"] = np.mean(prediction_lens)
    result["meteor"] = meteor_result["meteor"]
    result = {k: round(v, 4) for k, v in result.items()}
    return result

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [24]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

### Train and Save the Model

In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss


OSError: [Errno 28] No space left on device

In [ ]:
eval_result = trainer.evaluate(tokenized_datasets['test'])

In [ ]:
eval_result

In [ ]:
trainer.save_model()

In [ ]:
trainer.push_to_hub()

### Test

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
sentence = "开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快"

tokenizer.src_lang = 'zh_CexN'
tokenizer.tgt_lang = 'en_XX'

encoded = tokenizer(sentence, return_tensors="pt").to(device)
model = model.to(device)
generated_tokens = model.generate(**encoded)
decoded = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

print(decoded)

> "开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快"

> 'If the air conditioner is on, the flight resumes too quickly, especially in winter when the weather is cold. If the air conditioner is not on, the flight resumes faster as soon as the weather is cold'

In [ ]:
def translate(sentence):
    encoded = tokenizer(sentence, return_tensors="pt").to(device)
    generated_tokens = model.generate(**encoded)
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

In [ ]:
for i in dataset['test']['translation'][:5]:
    print("="*30)
    print(i['zh'])
    print(i['en'])
    print("="*10)
    result = translate(i['zh'])
    print(result)